In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from pyppeteer import launch
import random
import pandas as pd
from bs4 import BeautifulSoup
import re

def screen_size():
    import tkinter
    tk = tkinter.Tk()
    width = tk.winfo_screenwidth()
    height = tk.winfo_screenheight()
    tk.quit()
    return width, height

async def get_html(url):
    browser = await launch({'headless': False, 'userDataDir': r'D:/haruk/Documents/PythonCodes/Temp', 'args': ['--no-sandbox'], 'dumpio': True})
    page = await browser.newPage()
    width, height = screen_size()
    await page.setViewport(viewport={"width": width, "height": height})
    await page.setJavaScriptEnabled(enabled=True)
    await page.setUserAgent(
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
    )
    await page.goto(url)
    content_lst = []
    content_lst.append(await page.content())
    count = 0
    for i in range(1,18):
        try:
            # li_container.append(await page.xpath('//*[@id="li_container"]/li'))
            await page.evaluate('window.scrollBy(0, document.body.scrollHeight)')
            await asyncio.sleep(3)
            await page.click('#pagebar > a:nth-child({})'.format((i+3) if i<5 else 7))
            await asyncio.sleep(3)
            content_lst.append(await page.content())
        except:
            content_lst.append('Error @ iter {}'.format(i))
        finally:
            count += 1
            print('\r{:0>2d}/18'.format(count),end='')
    await asyncio.sleep(1)
    await page_close(browser)
    return content_lst

async def page_close(browser):
    for _page in await browser.pages():
        await _page.close()
    await browser.close()

def create_infoLst(html_lst):
    info_lst = []
    print('List creation Initiated.')
    for item in html_lst:
        dct = {}
        try:
            soup = BeautifulSoup(item, 'html.parser')
            tag1 = soup.find_all(name='a', attrs={'class':'title'})
            tag2 = soup.find_all(name="span", attrs={'class':"color_theme"})
            titles, durls, values, stvs, mxvs = [], [], [], [], []
            for tit in tag1:
                if not tit.string == None:
                    titles.append(str(tit.string))
                if not tit.attrs['href'] == None:
                    durls.append(tit.attrs['href'])
            for value in tag2:
                values.append(str(value.string))
            for i in range(len(values)):
                if i%2 == 0:
                    stvs.append(values[i])
                else:
                    mxvs.append(values[i])
        except:
            dct = {'title':[], 'deep_url':[], 'start_value':[], 'max_value':[]}
        else:
            dct = {'title':titles, 'deep_url':durls, 'start_value':stvs, 'max_value':mxvs}
        finally:
            info_lst.append(dct)
    return info_lst


'''
TESTING
'''

# p_url = 'https://jpxkc.cbex.com/jpxkc/zc_prjs/1064.html'
# loop = asyncio.get_event_loop()
# content_lst = loop.run_until_complete(get_html(p_url))
# info_lst = create_infoLst(content_lst)

In [ ]:
def check_result_list(res_lst):
    chklst = []
    count = 0
    saved = {'title':[0]}
    total = 0
    for item in res_lst:
        chklst.append([count]) # denoting a index for checklist
        idxx = False # check for redundent
        for title in item['title']:
            if title == saved['title'][0]:
                idxx = True
        if idxx:
            chklst[count].append('Redundant!!!')
        else:
            chklst[count].append('Verified')
        saved = item
        for key in item:
            chklst[count].append(len(item[key])) # check each string length
            chklst[count].append(str(type(item[key][0]))) # check for whether str type
        idx = False # check for no indented
        for each in chklst[count][2:9:2]:
            if not each == 16:
                idx = True
        if idx:
            chklst[count].append('Invalid')
        else:
            chklst[count].append('Ok')
        total += chklst[count][4] # statistic total numbers
        count += 1
    chklst.append(['total={}'.format(total)])
    return chklst

# chklst = check_result_list(info_lst)

In [ ]:
# save content_lst
def save_content_lst_to_html(content_lst):
    for i in range(len(content_lst)):
        with open('D:/haruk/Documents/PythonCodes/Downloads/carinfo_{:0>2d}.html'.format(i+1),
            'wt',encoding='utf-8') as f:
            try:
                stts = f.write(content_lst[i])
            except:
                print('No.{0:0>2d}: status:{1}; Error'.format(i+1, stts))
            else:
                print('No.{0:0>2d}: status:{1}; Success'.format(i+1, stts))
            finally:
                f.close()

# save_content_lst_to_html(content_lst)

In [ ]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(content_lst[1],'html.parser')
tag1 = soup.find_all(name='a', attrs={'class':'title'})
tag2 = soup.find_all(name="span", attrs={'class':"color_theme"})
titles, durls, values, stvs, mxvs = [], [], [], [], []
for tit in tag1:
    if not tit.string == None:
        titles.append(tit.string)
    if not tit.attrs['href'] == None:
        durls.append(tit.attrs['href'])
for value in tag2:
    values.append(value.string)
for i in range(len(values)):
    if i%2 == 0:
        stvs.append(values[i])
    else:
        mxvs.append(values[i])
dct = {'title':titles, 'deep_url':durls, 'start_value':stvs, 'max_value':mxvs}

# li_cont = testSoup.find(attrs={'id':'li_container'})
# carinfor = li_cont('li')
# carlst = []
# for item in carinfor:
#     carlst.append(item(string = re.compile(r'^京')))

In [ ]:
# save information list to excel:
def convert_infoLst_to_DataFrame(info_lst):
    # Initiate col names:
    col_names = []
    for key in info_lst[0]:
        col_names.append(key)
    # Initiate a DataFrame with col names:
    dtfm = pd.DataFrame(columns=col_names)
    pla_len = 0
    for i in range(len(info_lst)):
        pre_len = len(info_lst[i]['title'])
        pre_dtfm = pd.DataFrame(info_lst[i], index=[i for i in range(pla_len, pla_len + pre_len)])
        dtfm = dtfm.append(pre_dtfm)
        pla_len += pre_len
    return dtfm

# dtfm = convert_infoLst_to_DataFrame(info_lst1)

In [ ]:
# designed for split 'title', but useless now
def split_title(title_lst: list):
    result = []
    for item in title_lst:
        plate = item[:7]
        inx1 = int(re.search(r'[A-Z]',item[7:]).span()[0]) + 7
        brand = item[7:inx1]
        inx2 = int(re.search('[\u4E00-\u9FFF]',item[15:]).span()[0]) + 15
        carType = item[inx1:inx2]
        color = item[inx2:inx2+2]
        result.append([plate,brand,carType,color])
    return result